In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

Début 8h49

In [2]:
df = pd.read_csv("dtf_test.csv")

In [3]:
df.head()

,Unnamed: 0,date,hour,station_id,nbre_records,T2M,DEWT2M,RH2M,WS2M,WS10M,WD,VPD,CLDC,SSI,PRECIP,lat,lon,T2M_D_med,T2M_U_med
0,0,2020-03-15,1,1176,5,4.1,4.1,100.0,0.45,0.6,132.0,0.00,59.0,0.0,0.0,44.70687,-0.61908,0.3,0.1
1,1,2020-03-15,1,2030,6,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27178,-1.65603,9.8,7.8
2,2,2020-03-15,1,3149,4,3.9,3.6,98.0,0.97,1.3,121.0,0.02,52.0,0.0,0.0,44.82995,-0.18713,22.5,NaN
3,3,2020-03-15,1,3153,4,9.2,6.2,82.0,2.77,3.7,182.0,0.21,100.0,0.0,0.0,47.19588,-1.34525,12.1,NaN
4,4,2020-03-15,1,3154,5,9.1,7.5,90.0,1.87,2.5,184.0,0.12,100.0,0.0,0.0,47.09277,-1.04123,8.6,NaN


In [4]:
df=df.drop("Unnamed: 0", axis=1)

In [5]:
len(df)

11773387

**Suppression des valeurs extrêmes : températures (sèche ou humide) à exactement +60°C ou -40°C qui sont en fait des valeurs d’erreur du capteur**

In [6]:
df = df[(df["T2M_D_med"] != 60)]
df = df[(df["T2M_D_med"] != -40)]
df = df[df["T2M_U_med"] != 60]
df = df[df["T2M_U_med"] != -40]


In [7]:
len(df)

11754209

**Restriction à la période concernée par le gel : novembre - mai ;**

In [8]:
df["date"] = pd.to_datetime(df["date"])
df = df[(df["date"].dt.month >= 11) | (df["date"].dt.month <= 5)]

In [9]:
len(df)

7067490

**Suppression des données diurnes : SSI > 50 W/m²**

In [10]:
df = df[df["SSI"] <= 50]

In [11]:
len(df)

4205761

**Suppression des données où la température humide est supérieure à la température sèche : T2M_U > T2M_D + 0.5 (marge prenant en compte l’erreur de mesure)**

In [12]:
df = df[df["T2M_U_med"] <= (df["T2M_D_med"] + 0.5)]

In [13]:
len(df)

4079787

**Suppression des données très éloignées des situations de gel : T2M > 10°C**

In [14]:
df = df[df["T2M"] <= 10]

In [15]:
len(df)

2454656

**Suppression des données où températures atmosphériques et températures sèches sont trop éloignées : |T2M - T2M_D - 1.5| > 3°C**

In [16]:
df = df[np.abs(df["T2M"] - df["T2M_D_med"] - 1.5) <= 3]

In [17]:
len(df)

1954853

**Suppression des données manquantes sur les variables T2M_U_med**

In [18]:
df = df[df['T2M_U_med'].notna()]

In [19]:
len(df)

1954853

**Suppression des jours où le capteur émet strictement moins de 50 fois (en théorie entre 96 et 240 mesures par jour)**

In [ ]:
df_grp = df.groupby(['station_id', 'date'])['nbre_records'].sum().reset_index(name='nombre_total_record')
df2 = df_grp[df_grp['nombre_total_record'] > 50]
df2

,station_id,date,nombre_total_record
9,1176,2020-03-26,61
12,1176,2020-03-29,54
13,1176,2020-03-30,77
16,1176,2020-04-02,54
27,1176,2020-11-04,76
...,...,...,...
218687,23157,2023-04-08,70
218694,23157,2023-04-16,79
218695,23157,2023-04-17,74
218696,23157,2023-04-18,60


In [21]:
df = df.merge(df2[['station_id', 'date']], on=['station_id', 'date'])

In [ ]:
df

,date,hour,station_id,nbre_records,T2M,DEWT2M,RH2M,WS2M,WS10M,WD,VPD,CLDC,SSI,PRECIP,lat,lon,T2M_D_med,T2M_U_med
0,2020-03-15,1,2030,6,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27178,-1.65603,9.80,7.80
1,2020-03-15,1,3324,4,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27163,-1.65622,10.40,10.70
2,2020-03-15,1,3898,5,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27177,-1.65600,9.40,7.50
3,2020-03-15,1,3899,6,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27172,-1.65615,9.50,7.70
4,2020-03-15,1,4129,5,9.7,6.4,80.0,2.39,3.2,195.0,0.24,99.0,0.0,0.0,47.81595,-0.99628,8.80,7.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266682,2023-05-18,24,14671,4,8.8,3.6,70.0,0.82,1.1,12.0,0.34,0.0,0.0,0.0,45.44711,1.00071,6.60,6.20
1266683,2023-05-18,24,19156,4,8.2,4.1,75.0,2.92,3.9,358.0,0.27,0.0,0.0,0.0,47.78041,3.78086,9.05,7.60
1266684,2023-05-18,24,19761,4,6.6,5.0,90.0,1.57,2.1,132.0,0.10,96.0,0.0,0.0,43.98228,5.47658,6.90,6.90
1266685,2023-05-18,24,20475,11,8.1,6.0,87.0,1.27,1.7,94.0,0.14,0.0,0.0,0.0,47.67515,1.76627,5.00,4.60


**Suppression des données de capteurs en cours de fabrication, calibration, test et/ou réparation : stations situées à moins de 0.05° d’un point basé à Nantes de latitude 47.266 et de longitude -1.561.**

In [ ]:
lat, long = 47.266, -1.561
dist = (np.sqrt((df['lat'] - lat)**2 + (df['lon'] - long)**2) > 0.05)
df = df[dist]

In [24]:
df

,date,hour,station_id,nbre_records,T2M,DEWT2M,RH2M,WS2M,WS10M,WD,VPD,CLDC,SSI,PRECIP,lat,lon,T2M_D_med,T2M_U_med
0,2020-03-15,1,2030,6,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27178,-1.65603,9.80,7.80
1,2020-03-15,1,3324,4,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27163,-1.65622,10.40,10.70
2,2020-03-15,1,3898,5,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27177,-1.65600,9.40,7.50
3,2020-03-15,1,3899,6,9.8,4.5,70.0,3.29,4.4,198.0,0.36,27.0,0.0,0.0,47.27172,-1.65615,9.50,7.70
4,2020-03-15,1,4129,5,9.7,6.4,80.0,2.39,3.2,195.0,0.24,99.0,0.0,0.0,47.81595,-0.99628,8.80,7.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266682,2023-05-18,24,14671,4,8.8,3.6,70.0,0.82,1.1,12.0,0.34,0.0,0.0,0.0,45.44711,1.00071,6.60,6.20
1266683,2023-05-18,24,19156,4,8.2,4.1,75.0,2.92,3.9,358.0,0.27,0.0,0.0,0.0,47.78041,3.78086,9.05,7.60
1266684,2023-05-18,24,19761,4,6.6,5.0,90.0,1.57,2.1,132.0,0.10,96.0,0.0,0.0,43.98228,5.47658,6.90,6.90
1266685,2023-05-18,24,20475,11,8.1,6.0,87.0,1.27,1.7,94.0,0.14,0.0,0.0,0.0,47.67515,1.76627,5.00,4.60


fin : 9h53

In [25]:
df.to_csv("df_clean.csv", index=False)

Temps : 1h04